# Lesson 4: Chaining Prompts for Agentic Reasoning

## Automated Claim Triage: From First-Notice to the Right Queue

In this hands-on exercise, you will build a prompt chain that extracts key fields from free-form auto-claim reports, assesses damage severity, and routes each claim to one of several queues — `glass`, `fast_track`, `material_damage`, or `total_loss` — with code-based gate checks at every step.

## Outline:

- Setup
- Sample FNOL (First Notice of Loss) Texts
- Stage I: Information Extraction
- Stage II: Severity Assessment
- Stage III: Queue Routing
- Review Outputs

## 📝 RESUMEN DE COMPLETACIÓN Y CORRECCIONES

### ✅ Qué se completó en este notebook:

Este notebook implementa un **sistema de cadena de prompts (prompt chaining)** para triaje automatizado de claims de seguros de auto. El sistema procesa reportes FNOL (First Notice of Loss) en texto libre y los enruta a colas de procesamiento.

### 🔧 Correcciones realizadas por Claude:

1. **Cell 9 - ClaimInformation**:
   - ✅ Completada la clase Pydantic con todos los campos requeridos
   - ✅ Corregido `min_items` → `min_length` (deprecado en Pydantic V2)
   - ✅ Completado el prompt `info_extraction_system_prompt`
   - ✅ Mejorado el prompt para evitar valores inválidos en `damage_area`

2. **Cell 10 - Gate Check 1**:
   - ✅ Completada la función `extract_claim_info` con validación Gate 1
   - ✅ Agregados comentarios explicativos en español

3. **Cell 13 - SeverityAssessment**:
   - ✅ Completado el prompt `severity_assessment_system_prompt`
   - ✅ Agregados comentarios sobre los rangos de costo

4. **Cell 14 - Gate Check 2**:
   - ✅ Completadas las validaciones de rango de costos para Minor ($100-$1000), Moderate ($1000-$5000), Major ($5000-$50000)
   - ✅ Agregado manejo de `None` para prevenir AttributeError
   - ✅ Agregados comentarios explicativos

5. **Cell 17 - ClaimRouting**:
   - ✅ Completado el prompt `queue_routing_system_prompt`
   - ✅ Agregadas reglas de enrutamiento claramente definidas

6. **Cell 18 - Gate Check 3**:
   - ✅ Completado el diccionario `routing_input` con toda la información necesaria
   - ✅ Agregado manejo de `None` para claim_info y severity_assessment
   - ✅ Agregados comentarios explicativos

7. **Cell 21 - DataFrame de resultados**:
   - ✅ Agregado manejo robusto de valores `None`
   - ✅ Agregado resumen de claims exitosos vs fallidos
   - ✅ Uso correcto de `.model_dump()` para Pydantic V2

### 🎯 Cómo funciona el sistema:

**STAGE 1: Extracción de Información**
- Extrae datos estructurados del texto FNOL
- Gate Check 1 valida el JSON y los tipos de datos

**STAGE 2: Evaluación de Severidad**
- Evalúa la severidad del daño (Minor/Moderate/Major)
- Estima el costo de reparación
- Gate Check 2 valida que el costo esté en el rango correcto

**STAGE 3: Enrutamiento a Colas**
- Determina la cola apropiada basándose en severidad y tipo de daño
- Colas: `glass`, `fast_track`, `material_damage`, `total_loss`
- Gate Check 3 valida que la cola sea válida

### 🛡️ Gate Checks (Validaciones de Código):

Los **gate checks** son validaciones de código que previenen la cascada de errores:
- Si un LLM devuelve datos inválidos, el gate check los detecta
- Previene que datos incorrectos pasen al siguiente stage
- Permite manejo elegante de errores sin romper el pipeline

---

## Setup

Import necessary libraries and define helper functions, including a mock LLM client, code execution environment, and test runner.

In [5]:
# Import necessary libraries
# No changes needed in this cell
from openai import OpenAI  # For accessing the OpenAI API
from enum import Enum
import json
from pydantic import BaseModel, Field  # For structured data validation
from typing import List, Literal, Optional
import os


In [ ]:
# Set up LLM credentials

client = OpenAI(
    base_url="https://openai.vocareum.com/v1",
    # Uncomment one of the following
    # api_key="**********",  # <--- TODO: Fill in your Vocareum API key here
    # api_key=os.getenv(
    #     "OPENAI_API_KEY"
    # ),  # <-- Alternately, set as an environment variable
)

# If using OpenAI's API endpoint
# client = OpenAI()

In [2]:
from dotenv import load_dotenv
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [6]:
# Define helper functions
# No changes needed in this cell


class OpenAIModels(str, Enum):
    GPT_4O_MINI = "gpt-4o-mini"
    GPT_41_MINI = "gpt-4.1-mini"
    GPT_41_NANO = "gpt-4.1-nano"


MODEL = OpenAIModels.GPT_41_NANO


def get_completion(messages=None, system_prompt=None, user_prompt=None, model=MODEL):
    """
    Function to get a completion from the OpenAI API.
    Args:
        system_prompt: The system prompt
        user_prompt: The user prompt
        model: The model to use (default is gpt-4.1-mini)
    Returns:
        The completion text
    """

    messages = list(messages)
    if system_prompt:
        messages.insert(0, {"role": "system", "content": system_prompt})
    if user_prompt:
        messages.append({"role": "user", "content": user_prompt})
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7,
    )
    return response.choices[0].message.content

## Sample FNOL (First Notice of Loss) Texts
Let's define a few sample First Notice of Loss (FNOL) texts to process through our chain.

In [13]:
# Define sample FNOL texts
# TODO: [Optional] Add more sample FNOL texts to test various scenarios

sample_fnols = [
    """
    Claim ID: C001
    Customer: John Smith
    Vehicle: 2018 Toyota Camry
    Incident: While driving on the highway, a rock hit my windshield and caused a small chip
    about the size of a quarter. No other damage was observed.
    """,
    """
    Claim ID: C002
    Customer: Sarah Johnson
    Vehicle: 2020 Honda Civic
    Incident: I was parked at the grocery store and returned to find someone had hit my car and
    dented the rear bumper and taillight. The taillight is broken and the bumper has a large dent.
    """,
    """
    Claim ID: C003
    Customer: Michael Rodriguez
    Vehicle: 2022 Ford F-150
    Incident: I was involved in a serious collision at an intersection. The front of my truck is
    severely damaged, including the hood, bumper, radiator, and engine compartment. The airbags
    deployed and the vehicle is not drivable.
    """,
    """
    Claim ID: C004
    Customer: Emma Williams
    Vehicle: 2019 Subaru Outback
    Incident: My car was damaged in a hailstorm. There are multiple dents on the hood, roof, and
    trunk. The side mirrors were also damaged and one window has a small crack.
    """,
    """
    Claim ID: C005
    Customer: David Brown
    Vehicle: 2021 Tesla Model 3
    Incident: Someone keyed my car in the parking lot. There are deep scratches along both doors
    on the driver's side.
    """,
]

## Stage I: Information Extraction
In this stage, we'll create a prompt that extracts structured information from free-form FNOL text.

In [ ]:
# ========================================
# STAGE I: EXTRACCIÓN DE INFORMACIÓN
# ========================================
# PROPÓSITO: Extraer datos estructurados de reportes FNOL en texto libre
# COMPLETADO POR CLAUDE: Se completó la clase ClaimInformation y el prompt de extracción

class ClaimInformation(BaseModel):
    """
    Modelo Pydantic que define la estructura de información de un claim.
    Valida que los datos extraídos cumplan con los requisitos:
    - claim_id: ID único del claim (2-10 caracteres)
    - name: Nombre completo del cliente (2-100 caracteres)
    - vehicle: Descripción del vehículo (2-100 caracteres)
    - loss_desc: Descripción del incidente (10-500 caracteres)
    - damage_area: Lista de áreas dañadas (mínimo 1 área)
    """
    claim_id: str = Field(..., min_length=2, max_length=10)
    name: str = Field(..., min_length=2, max_length=100)
    vehicle: str = Field(..., min_length=2, max_length=100)
    loss_desc: str = Field(..., min_length=10, max_length=500)
    damage_area: List[
        Literal[
            "windshield",
            "front",
            "rear",
            "side",
            "roof",
            "hood",
            "door",
            "bumper",
            "fender",
            "quarter panel",
            "trunk",
            "glass",
        ]
    ] = Field(..., min_length=1)  # CORREGIDO: Cambiado de min_items a min_length


info_extraction_system_prompt = """
You are an expert insurance claims processor. Your task is to extract key information from First Notice of Loss (FNOL) reports.

IMPORTANT: For damage_area, you MUST ONLY use these exact values:
- windshield, front, rear, side, roof, hood, door, bumper, fender, quarter panel, trunk, glass

Do NOT invent new values or use variations (e.g., "taillight" should map to "rear" or "glass", not "light").

Format your response as a valid JSON object with the following keys:
- claim_id (str): The claim ID
- name (str): The customer's full name
- vehicle (str): The vehicle make, model, and year
- loss_desc (str): A description of the loss or incident
- damage_area (list): A list of damaged areas using ONLY the allowed values above

Extract the information accurately from the FNOL text. For damage_area, identify all applicable areas mentioned in the incident description.

Only respond with the JSON object, nothing else.
"""

In [ ]:
# ========================================
# GATE CHECK 1 y FUNCIÓN DE EXTRACCIÓN
# ========================================
# COMPLETADO POR CLAUDE: Se completó la función extract_claim_info con validación Gate 1

def gate1_validate_claim_info(claim_info_json: str) -> ClaimInformation:
    """
    GATE CHECK 1: Valida la información extraída del claim
    
    PROPÓSITO: Asegurar que el LLM devolvió JSON válido con todos los campos requeridos
    y que cumplen con las restricciones de Pydantic (longitudes, tipos, valores literales)
    
    Returns:
        ClaimInformation validado
    Raises:
        ValueError si la validación falla
    """
    try:
        # Parse the JSON string
        claim_info_dict = json.loads(claim_info_json)
        # Validate with Pydantic model
        validated_info = ClaimInformation(**claim_info_dict)
        return validated_info
    except Exception as e:
        raise ValueError(f"Gate 1 validation failed: {str(e)}")


def extract_claim_info(fnol_text):
    """
    STAGE 1: Extraer información estructurada del texto FNOL
    
    FLUJO:
    1. Envía el texto FNOL al LLM con el prompt de extracción
    2. Recibe respuesta JSON del LLM
    3. Valida la respuesta con Gate Check 1
    4. Retorna ClaimInformation validado o None si falla
    """
    messages = [
        {"role": "system", "content": info_extraction_system_prompt},
        {"role": "user", "content": fnol_text},
    ]

    response = get_completion(messages=messages)

    # Gate check: validate the extracted information
    try:
        validated_info = gate1_validate_claim_info(response)  # <-- COMPLETADO: Validación Gate 1
        return validated_info
    except ValueError as e:
        print(f"Gate 1 failed: {e}")
        return None

In [14]:
# Run the claim extraction function on the sample FNOLs
# No updates needed in this cell

extracted_claim_info_items = [
    extract_claim_info(fnol_text) for fnol_text in sample_fnols
]
extracted_claim_info_items

Gate 1 failed: Gate 1 validation failed: 1 validation error for ClaimInformation
damage_area.2
  Input should be 'windshield', 'front', 'rear', 'side', 'roof', 'hood', 'door', 'bumper', 'fender', 'quarter panel', 'trunk' or 'glass' [type=literal_error, input_value='light', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/literal_error


[ClaimInformation(claim_id='C001', name='John Smith', vehicle='2018 Toyota Camry', loss_desc='While driving on the highway, a rock hit my windshield and caused a small chip about the size of a quarter. No other damage was observed.', damage_area=['windshield']),
 None,
 ClaimInformation(claim_id='C003', name='Michael Rodriguez', vehicle='2022 Ford F-150', loss_desc='Serious collision at an intersection with severe damage to the front, including hood, bumper, radiator, and engine compartment. Airbags deployed, vehicle not drivable.', damage_area=['front', 'hood', 'bumper']),
 ClaimInformation(claim_id='C004', name='Emma Williams', vehicle='2019 Subaru Outback', loss_desc='My car was damaged in a hailstorm. There are multiple dents on the hood, roof, and trunk. The side mirrors were also damaged and one window has a small crack.', damage_area=['hood', 'roof', 'trunk', 'side', 'glass']),
 ClaimInformation(claim_id='C005', name='David Brown', vehicle='2021 Tesla Model 3', loss_desc="Someon

## Stage II: Severity Assessment
In this stage, we'll assess the severity of the damage based on the extracted information.

Note, our carrier applies the following heuristics:
- Minor damage: Small dents, scratches, glass chips (cost range: $100-$1,000)
- Moderate damage: Single panel damage, bumper replacement, door damage (cost range: $1,000-$5,000)
- Major damage: Structural damage, multiple panel replacement, engine/drivetrain issues, total loss candidates (cost range: $5,000-$50,000)

In this example we will let the LLM estimate the cost, though in production we would want a more accurate estimate, e.g. querying a database of repair costs.

In [ ]:
# ========================================
# STAGE II: EVALUACIÓN DE SEVERIDAD
# ========================================
# PROPÓSITO: Evaluar la severidad del daño y estimar el costo de reparación
# COMPLETADO POR CLAUDE: Se completó el prompt de evaluación de severidad

class SeverityAssessment(BaseModel):
    """
    Modelo Pydantic para la evaluación de severidad del daño
    
    - severity: Nivel de severidad (Minor, Moderate, Major)
    - est_cost: Costo estimado de reparación en dólares (debe ser > 0)
    """
    severity: Literal["Minor", "Moderate", "Major"]
    est_cost: float = Field(..., gt=0)


severity_assessment_system_prompt = """
You are an auto insurance damage assessor. Your task is to evaluate the severity of vehicle damage and estimate repair costs.

Based on the claim information provided, assess the severity and estimated repair cost:
- Minor: Small dents, scratches, glass chips (cost range: $100-$1,000)
- Moderate: Single panel damage, bumper replacement, door damage (cost range: $1,000-$5,000)
- Major: Structural damage, multiple panel replacement, engine/drivetrain issues, total loss candidates (cost range: $5,000-$50,000)

IMPORTANT: Your estimated cost (est_cost) MUST fall within the appropriate range for the severity level you assign.

Format your response as a valid JSON object with the following keys:
- severity (str): Must be one of "Minor", "Moderate", or "Major"
- est_cost (float): Estimated repair cost in dollars (must be within the range for the severity)

Only respond with the JSON object, nothing else.
"""

In [ ]:
# ========================================
# GATE CHECK 2 y FUNCIÓN DE EVALUACIÓN DE SEVERIDAD
# ========================================
# COMPLETADO POR CLAUDE: Se completaron las validaciones de rango de costos

def gate2_cost_range_ok(severity_json: str) -> SeverityAssessment:
    """
    GATE CHECK 2: Valida que el costo estimado esté dentro del rango correcto para la severidad
    
    PROPÓSITO: Prevenir que el LLM asigne costos inconsistentes con la severidad
    (ej: "Minor" con costo de $10,000)
    
    RANGOS VALIDADOS:
    - Minor: $100 - $1,000
    - Moderate: $1,000 - $5,000
    - Major: $5,000 - $50,000
    
    Returns:
        SeverityAssessment validado
    Raises:
        ValueError si el costo está fuera del rango
    """
    try:
        # Parse the JSON string
        severity_dict = json.loads(severity_json)
        # Validate with Pydantic model
        validated_severity = SeverityAssessment(**severity_dict)

        # Check cost range based on severity
        if (
            validated_severity.severity == "Minor"
            and (
                validated_severity.est_cost < 100 or validated_severity.est_cost > 1000
            )
        ):
            raise ValueError(
                f"Minor damage should cost between $100-$1000, got ${validated_severity.est_cost}"
            )
        elif (
            validated_severity.severity == "Moderate"
            and (
                validated_severity.est_cost < 1000 or validated_severity.est_cost > 5000
            )
        ):
            raise ValueError(
                f"Moderate damage should cost between $1000-$5000, got ${validated_severity.est_cost}"
            )
        elif (
            validated_severity.severity == "Major"
            and (
                validated_severity.est_cost < 5000 or validated_severity.est_cost > 50000
            )
        ):
            raise ValueError(
                f"Major damage should cost between $5000-$50000, got ${validated_severity.est_cost}"
            )

        return validated_severity
    except Exception as e:
        raise ValueError(f"Gate 2 validation failed: {str(e)}")


def assess_severity(claim_info: ClaimInformation) -> Optional[SeverityAssessment]:
    """
    STAGE 2: Evaluar la severidad del daño basándose en la información del claim
    
    FLUJO:
    1. Convierte ClaimInformation a JSON
    2. Envía al LLM para evaluación de severidad
    3. Valida respuesta con Gate Check 2
    4. Retorna SeverityAssessment validado o None si falla
    
    Args:
        claim_info: Información validada del claim (o None si Stage 1 falló)
    Returns:
        SeverityAssessment o None
    """
    # CORREGIDO: Manejo de None - si el claim no se extrajo correctamente, retornar None
    if claim_info is None:
        return None
    
    # Convert Pydantic model to JSON string
    claim_info_json = claim_info.model_dump_json()

    messages = [
        {"role": "system", "content": severity_assessment_system_prompt},
        {"role": "user", "content": claim_info_json},
    ]

    response = get_completion(messages=messages)

    # Gate check: validate the severity assessment
    try:
        validated_severity = gate2_cost_range_ok(response)
        return validated_severity
    except ValueError as e:
        print(f"Gate 2 failed: {e}. Response: {response}")
        return None

In [17]:
# Run the claim extraction function on the sample data
# No updates needed in this cell

severity_assessment_items = [
    assess_severity(item) for item in extracted_claim_info_items
]

severity_assessment_items

AttributeError: 'NoneType' object has no attribute 'model_dump_json'

## Stage III: Queue Routing
In this stage, we'll route the claim to the appropriate queue based on severity and damage area.

Use these routing rules:
- 'glass' queue: For Minor damage involving ONLY glass (windshield, windows)
- 'fast_track' queue: For other Minor damage
- 'material_damage' queue: For all Moderate damage
- 'total_loss' queue: For all Major damage

In [ ]:
# ========================================
# STAGE III: ENRUTAMIENTO A COLAS
# ========================================
# PROPÓSITO: Determinar la cola de procesamiento apropiada basándose en severidad y tipo de daño
# COMPLETADO POR CLAUDE: Se completó el prompt de enrutamiento

class ClaimRouting(BaseModel):
    """
    Modelo Pydantic para el enrutamiento del claim a la cola apropiada
    
    - claim_id: ID del claim
    - queue: Cola asignada (glass, fast_track, material_damage, total_loss)
    """
    claim_id: str
    queue: Literal["glass", "fast_track", "material_damage", "total_loss"]


queue_routing_system_prompt = """
You are an auto insurance claim routing specialist. Your task is to determine the appropriate processing queue for each claim.

ROUTING RULES (MUST FOLLOW STRICTLY):
1. 'glass' queue: For Minor damage involving ONLY glass (windshield, windows)
2. 'fast_track' queue: For other Minor damage (not glass-only)
3. 'material_damage' queue: For ALL Moderate damage
4. 'total_loss' queue: For ALL Major damage

IMPORTANT: Analyze both the severity level AND the damage_area list to determine the correct queue.
For glass queue, the damage must be Minor AND only affect glass areas (windshield, glass).

Format your response as a valid JSON object with the following keys:
- claim_id (str): The claim ID from the claim information
- queue (str): Must be one of "glass", "fast_track", "material_damage", or "total_loss"

Only respond with the JSON object, nothing else.
"""

In [ ]:
# ========================================
# GATE CHECK 3 y FUNCIÓN DE ENRUTAMIENTO
# ========================================
# COMPLETADO POR CLAUDE: Se completó la función route_claim

def gate3_validate_routing(routing_json: str) -> ClaimRouting:
    """
    GATE CHECK 3: Valida que el claim se haya enrutado a una cola válida
    
    PROPÓSITO: Asegurar que el LLM devolvió un JSON válido con claim_id y 
    una cola válida (glass, fast_track, material_damage, total_loss)
    
    Returns:
        ClaimRouting validado
    Raises:
        ValueError si la validación falla
    """
    try:
        # Parse the JSON string
        routing_dict = json.loads(routing_json)
        # Validate with Pydantic model
        validated_routing = ClaimRouting(**routing_dict)
        return validated_routing
    except Exception as e:
        raise ValueError(f"Gate 3 validation failed: {str(e)}")


def route_claim(
    claim_info: ClaimInformation, severity_assessment: Optional[SeverityAssessment]
) -> Optional[ClaimRouting]:
    """
    STAGE 3: Enrutar el claim a la cola de procesamiento apropiada
    
    FLUJO:
    1. Verifica que tengamos información válida (claim_info y severity_assessment)
    2. Combina toda la información en un diccionario
    3. Envía al LLM para determinar la cola apropiada
    4. Valida respuesta con Gate Check 3
    5. Retorna ClaimRouting validado o None si falla
    
    Args:
        claim_info: Información del claim (o None si Stage 1 falló)
        severity_assessment: Evaluación de severidad (o None si Stage 2 falló)
    Returns:
        ClaimRouting o None
    """
    # CORREGIDO: Si cualquier stage previo falló, no podemos enrutar
    if claim_info is None or severity_assessment is None:
        return None

    # Create input for the routing model - combina información de Stage 1 y Stage 2
    routing_input = {
        "claim_id": claim_info.claim_id,
        "name": claim_info.name,
        "vehicle": claim_info.vehicle,
        "loss_desc": claim_info.loss_desc,
        "damage_area": claim_info.damage_area,
        "severity": severity_assessment.severity,
        "est_cost": severity_assessment.est_cost
    }

    messages = [
        {"role": "system", "content": queue_routing_system_prompt},
        {"role": "user", "content": json.dumps(routing_input)},
    ]

    response = get_completion(messages=messages)

    # Gate check: validate the routing decision
    try:
        validated_routing = gate3_validate_routing(response)
        return validated_routing
    except ValueError as e:
        print(f"Gate 3 failed: {e}. Response: {response}")
        return None

In [ ]:
# Run the routing function on the sample data
# No updates needed in this cell

routed_claim_items = [
    route_claim(claim, severity_assessment)
    for claim, severity_assessment in zip(
        extracted_claim_info_items, severity_assessment_items
    )
]

routed_claim_items

## Review Outputs

Let's put our data into a pandas dataframe for easier analysis.

In [ ]:
# ========================================
# REVISIÓN DE RESULTADOS: Crear DataFrame
# ========================================
# PROPÓSITO: Combinar todos los resultados en un DataFrame de pandas para análisis
# CORREGIDO POR CLAUDE: Agregado manejo de None values

import pandas as pd

records = []
for claim, severity_assessment, routed_claim in zip(
    extracted_claim_info_items, severity_assessment_items, routed_claim_items
):
    record = {}
    
    # CORREGIDO: Solo agregar datos si no son None
    # Si algún stage falló, el resultado será None y lo manejamos aquí
    if claim is not None:
        record.update(claim.model_dump())  # Convertir Pydantic a dict
    else:
        record['claim_id'] = 'FAILED_EXTRACTION'
        record['name'] = None
        record['vehicle'] = None
        record['loss_desc'] = None
        record['damage_area'] = None
    
    if severity_assessment is not None:
        record.update(severity_assessment.model_dump())  # Convertir Pydantic a dict
    else:
        record['severity'] = None
        record['est_cost'] = None
    
    if routed_claim is not None:
        record.update(routed_claim.model_dump())  # Convertir Pydantic a dict
    else:
        record['queue'] = None
    
    records.append(record)

# Show the entire dataframe since it is not too large
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
df = pd.DataFrame(records)

print("\n" + "="*80)
print("RESULTADOS FINALES DEL PIPELINE DE CHAINING PROMPTS")
print("="*80)
print(f"\nTotal de claims procesados: {len(df)}")
print(f"Claims exitosos (completaron todos los stages): {df['queue'].notna().sum()}")
print(f"Claims fallidos: {df['queue'].isna().sum()}")
print("\n" + "="*80 + "\n")

df

## Summary

🎉 Congratulations! 🎉 You've built an impressive prompt chain system for insurance claims!
You transformed messy FNOL text into structured data, assessed damage severity, and routed claims to the right queues, all with robust gate checks! 🚀✨

Remember:

- 🔗 Chained prompts break complex tasks into manageable steps
- 🛡️ Gate checks prevent error cascades
- 🧠 Having specialized prompts helps keep code focused and maintainable

You've mastered a powerful pattern for countless business processes! 🏆
Amazing work on your agentic reasoning system! 💯